In [44]:
from finvizfinance.screener.overview import Overview
import pandas as pd
import warnings
import certifi
import json
from urllib.request import urlopen
warnings.filterwarnings("ignore")
from datetime import datetime
today = datetime.today().strftime('%Y_%m_%d')

In [2]:
stock_scores = pd.read_csv('stock_scores/stock_score_data_recent.csv')

In [35]:
filters_dict = {'Debt/Equity':'Over 1',
                    #'Operating Margin':'Negative (<0%)', 
                    'P/E': 'Over 5'}

In [36]:
foverview = Overview()
foverview.set_filter(filters_dict=filters_dict)
df_overview = foverview.screener_view()
tickers = df_overview['Ticker'].to_list()

In [18]:
non_value_scores = stock_scores[stock_scores['ticker'].isin(tickers)]
non_value_scores = non_value_scores[['ticker', '2023', 'roce_rank', 'coef_rank', 'std_rank', 'final_rank']]

In [19]:
non_value_scores

,ticker,2023,roce_rank,coef_rank,std_rank,final_rank
271,KMX,0.044171,1334.0,1390.0,372.0,1496.0
864,CCOI,-0.020688,1928.0,1255.0,1124.0,1608.0
1530,LFCR,-0.199282,1947.0,1582.0,1387.0,1683.0
1885,PI,-0.132296,924.0,1128.0,1737.0,1006.0


In [21]:
filters_dict = {'20-Day Simple Moving Average': 'Price below SMA20', 
                '50-Day Simple Moving Average' : 'SMA50 above SMA20',
                #'200-Day Simple Moving Average': 'SMA200 below SMA50',
               "Index": 'Any'} #'RUSSELL 2000'
foverview = Overview()
foverview.set_filter(filters_dict=filters_dict)
df_overview = foverview.screener_view()
trending_stocks = df_overview['Ticker'].to_list()

In [45]:
final_stocks = non_value_scores[non_value_scores['ticker'].isin(trending_stocks)].sort_values(by='final_rank')
short_stocks = final_stocks[final_stocks['final_rank'] >= 1200]
#get current price of stocks
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)
    
def get_price(tick):
    url = (f"https://financialmodelingprep.com/api/v3/quote/{tick}?apikey=PHaANSXTwW2zC5hpGFO1uhe8EkPXgio7")
    data = get_jsonparsed_data(url)
    price = data[0]['price']
    return price

short_stocks['price_picked'] = short_stocks['ticker'].apply(get_price)

print(short_stocks)

    ticker      2023  roce_rank  coef_rank  std_rank  final_rank  price_picked
271    KMX  0.044171     1334.0     1390.0     372.0      1496.0            72


In [46]:
short_stocks['date'] = today
#short_stocks.to_csv('short_stocks.csv',  index=False)
short_stocks.to_csv('short_stocks.csv', mode='a', header=False, index=False)